# Computing the volume underneath a Pareto frontier
Given a Pareto-optimal frontier of solutions to a multi-criterion optimization model, this program computes the Pearson correlation coefficients between each pair of objectives. This provides a measure of the conflict among the objectives.

In [2]:
import pandas as pd
import numpy as np
import itertools

### Prepocess data and normalize the objective space

In [57]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("../solutionSets/3d/ClimateChange_None/climateChange_EfficientSolutions_NoneOnly.csv")

In [58]:
# Hard-coded manipulations to tidy-up sols

sols = sols.drop(["SolutionIndex","Frontier"],axis=1) #remove solution index and frontier columns (climate change sets)
#sols = sols.drop(["SolutionIndex"],axis=1) #remove solution index column (pack forest and Chilean sets and portfolio opt)
# nothing for some sets (toth mcDill, small sed fire)

In [59]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [60]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the sols dataframe

objSenses = {}

In [61]:
# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 0
objSenses["Northern Spotted Owl Habitat (ha)"] = 0 #for climate change sets (all others 0)
#objSenses["PER"] = 0 # for tothMcDill set (all others 1)
#objSenses['MaxSediment'] = 0
#objSenses[" Risk"] = 0
#All 1 for PackForest sets and Chilean reserve

In [62]:
# Get the objectives' bounds

objBounds = {}
for obj,sense in objSenses.items():
    objBounds[obj] = [sols[obj].min(),sols[obj].max()]

In [63]:
# Get each objective's ideal value

idealObjVals = {}
for obj,bounds in objBounds.items():
    idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max

In [64]:
# Get each objective's worst value

worstObjVals = {}
for obj,bounds in objBounds.items():
    worstObjVals[obj] = bounds[not(objSenses[obj])]

In [65]:
# Compute the non-normalized volume of the objective space

objSpaceVolume_nonNorm = 1
for obj,bounds in objBounds.items():
    objSpaceVolume_nonNorm *= bounds[1] - bounds[0]

In [66]:
# Normalize the objective space by converting each value to the relative achievement along its axis:

#   distance from the worst case value
# ---------------------------------------
# total distance spanned by the objective

# Objective space is now in [0,1]^N where N is number of objectives


for obj,bounds in objBounds.items():
    sols[obj] = sols[obj].apply(
        lambda x: \
        abs(x - worstObjVals[obj]) / \
        (bounds[1]-bounds[0]))

### Computing the spacing of solutions along the frontier

In [67]:
# Compute the d_i's and average dbar


sols["di"] = np.infty
for idx,row in sols.iterrows():
    minDist = sols.ix[idx]["di"]
    for idx2,row2 in sols.iterrows():
        if idx2 == idx:
            continue
        dist = np.linalg.norm(row[objs]-row2[objs])
        if dist < minDist:
            minDist = dist
    sols.set_value(idx,"di",minDist)
dbar = np.mean(sols["di"])

In [68]:
# Compute the spacing metric


spacing = np.sqrt(sum(np.power(sols["di"]-dbar,2))/(len(sols)-1))
spacing

0.063986660439831852